In [4]:
import json
from google import genai
from google.genai import types
import pathlib
import os

from phase2_subAgent.Check_safety import llm_moderator_check_safety
from phase2_subAgent.Eval_assess_step import llm_evaluator_assess_step
from phase2_subAgent.Eval_assess_closing import llm_evaluator_assess_closing
from phase2_subAgent.Gen_intro import llm_moderator_generate_intro
from phase2_subAgent.Generate_Summary import llm_moderator_generate_summary
from phase2_subAgent.Debater_Generate_Attack import llm_debater_generate_attack



In [5]:
def run_debate_session_logic(session_profile, lecture_material):

    # 1. 설정값 파싱
    dialogue_style = session_profile.get('mode_configuration', {}).get('dialogue_style')

    # 학습 주제 및 키워드 추출
    topic_info = session_profile.get('content_context', {}).get('target_topic', {})
    topic_keyword = topic_info.get('keyword')
    topic_description = topic_info.get('description')

    # 규칙 및 난이도 설정
    rules = session_profile.get('session_rules', {})
    max_turns = rules.get('max_turns')
    difficulty = rules.get('difficulty_parameter', {}).get('level')

    # 2. 상태 변수 초기화
    history = []
    evaluation_logs = []
    current_score = 50
    turn_count = 0  # 0부터 시작 (실제 턴 표기는 +1 하여 사용)
    WINNING_THRESHOLD = 70

    retry_count = 0
    MAX_RETRIES = 3

    print("=== [", dialogue_style, "] 세션을 시작합니다. (난이도:", difficulty, ") ===")

    # 3. 오프닝 (사회자 LLM)
    intro_response_str = llm_moderator_generate_intro(topic_keyword, rules, topic_description, dialogue_style)
    intro_response = json.loads(intro_response_str)
    intro_message = intro_response.get("message")

    print("\n[사회자]:", intro_message)

    history.append({"role": "system", "content": intro_message, "turn": "Opening"})

    user_input = input("\n[나 (입론)]: ")

    # 4. 메인 토론 루프
    while turn_count < max_turns:

        # Step 4-1. 안전성 검사
        safety_check_str = llm_moderator_check_safety(user_input, topic_keyword, topic_description)
        safety_check = json.loads(safety_check_str)

        if safety_check.get('is_valid') == False:
            retry_count += 1

            warning_msg = safety_check.get('reason')
            print("([사회자]: 경고!", warning_msg, "(감점 -5점)")

            if retry_count >= MAX_RETRIES:
                print(f"\n[사회자]: 부적절한 입력이 {MAX_RETRIES}회 반복되어 세션을 강제 종료합니다.")
                final_status = "ABORT"
                break

            current_score = max(0, current_score - 5)
            user_input = input(f"[나 (다시 입력) - {retry_count}/{MAX_RETRIES}]: ")
            continue

        retry_count = 0

        current_turn_display = turn_count + 1
        history.append({"role": "user", "content": user_input, "turn": current_turn_display})

        # Step 4-2. 논리 평가
        last_attack = history[-2]['content'] if len(history) > 2 else "Initial Argument"
        eval_result_str = llm_evaluator_assess_step(user_input, last_attack, lecture_material, dialogue_style, difficulty)
        eval_result = json.loads(eval_result_str)

        score_change = eval_result.get('score_delta', 0)
        current_score = max(0, min(100, current_score + score_change))

        evaluation_logs.append({
            "turn": current_turn_display,  
            "score_now": current_score,
            "reason": eval_result.get('rationale', '')
        })

        # Step 4-3. 콜드 게임 체크
        if current_score <= 20:
            print("\n[사회자]: 점수가 너무 낮아 더 이상 토론 진행이 어렵습니다. (Cold Game)")
            final_status = "COLD_GAME"
            break

        # Step 4-4. 반박 공격
        debater_response_str = llm_debater_generate_attack(user_input, history, lecture_material, difficulty, dialogue_style)
        debater_response = json.loads(debater_response_str)
        debater_msg = debater_response.get('argument', '')

        print("\n[Debater]:", debater_msg)

        history.append({"role": "assistant", "content": debater_msg, "turn": current_turn_display})

        # Step 4-5. 다음 턴 준비
        turn_count += 1

        if turn_count < max_turns:
            user_input = input(f"\n[나 (반론 {turn_count + 1}/{max_turns})]: ")

    # 5. 마무리 단계
    if 'final_status' not in locals():
        print("\n[사회자]: 정해진 토론 시간이 끝났습니다. 최후 변론을 해주세요.")

        final_user_speech = input("[나 (최종 발언)]: ")
        
        history.append({"role": "user", "content": final_user_speech, "turn": "Final"})
        # Step 5-1. 최종 평가
        final_eval_str = llm_evaluator_assess_closing(final_user_speech, history, lecture_material, difficulty, dialogue_style)
        final_eval = json.loads(final_eval_str)
        current_score = max(0, min(100, current_score + final_eval.get('score_delta', 0)))

        # 최종 평가 로그 추가
        evaluation_logs.append({
            "turn": "Final",
            "user_speech": final_user_speech,
            "score_now": current_score,
            "reason": final_eval.get('final_impression', '')
        })

        # Step 5-2. 승패 판정
        if current_score >= WINNING_THRESHOLD:
            final_status = "WIN"
        else:
            final_status = "LOSS"

        # Step 5-3. 결과 요약
        summary_context_str = llm_moderator_generate_summary(history, evaluation_logs, final_status)
        summary_context = json.loads(summary_context_str)

        print("\n" + "="*30)
        print("    결과:", final_status, "(최종 점수:", current_score, "점)")
        print("="*30)
        print("[총평]:", summary_context.get('summary_text', ''))

    # 6. 최종 결과 반환
    if 'summary_context' in locals(): 
        return final_status, evaluation_logs, summary_context, history
    else: 
        # 강제 종료된 경우 summary_context가 없음 -> 이를 처리하기 위함
        return final_status, evaluation_logs, None, history

In [6]:
# 테스트 실행
session_profile = {
  "mode_configuration": {
    "dialogue_style": "Debate_Mode",
    "interaction_goal": "Concept_Verification",
    "goal_description": "교과서적 개념에 대해 사용자가 설명하고, 상대 에이전트가 이를 반박하거나 허점을 지적하며 개념의 정확성을 검증함"
  },
  "content_context": {
    "target_topic": {
      "keyword": "DQN (Deep Q-Network)",
      "description": "DQN의 주요 메커니즘인 Experience Replay와 Target Network의 필요성에 대한 개념적 방어"
    },
    "knowledge_boundary": "Lecture_Only"
  },
  "session_rules": {
    "max_turns": 5,
    "difficulty_parameter": {
      "level": "High",
      "custom_constraints": [
        "논리적 비약이 있을 경우 즉시 지적할 것",
        "수식보다는 개념의 인과관계 위주로 반박할 것"
      ]
    }
  }
}

lecture_material = "없습니다, 강의 자료가 특별히 없어요"

final_status, evaluation_logs, summary_context, history = run_debate_session_logic(session_profile, lecture_material)

=== [ Debate_Mode ] 세션을 시작합니다. (난이도: High ) ===

[사회자]: 지금부터 DQN(Deep Q-Network)의 핵심 메커니즘인 'Experience Replay'와 'Target Network'의 필연성에 대한 심화 비판 토론을 시작하겠습니다. 본 세션은 총 5턴 동안 진행되며, 난이도 '상'의 엄격한 기준으로 당신의 논리를 검증할 것입니다. 수식의 단순 나열보다는 개념 간의 인과관계를 중점적으로 평가하며, 작은 논리적 비약도 즉각적인 반박의 대상이 됩니다. 강화학습의 고질적인 불안정성 문제를 해결하기 위해 왜 이 두 기능이 필수적인지, 그 당위성을 입증하며 첫 발언을 시작해 주십시오.
([사회자]: 경고! DQN의 주요 메커니즘에 대해 구체적인 주장이나 의견을 제시해 주셔야 토론을 진행할 수 있습니다. (감점 -5점)

[Debater]: i.i.d. 가정을 충족시키기 위해 Experience Replay가 '필수적'이라고 하셨는데, 그렇다면 별도의 버퍼 없이 병렬 에이전트를 활용해 데이터 상관관계를 깨는 A3C 같은 비동기 방식은 어떻게 설명하실 겁니까? 또한, Replay Buffer에서 샘플링된 데이터는 결국 과거 정책에 의해 수집된 것이므로 현재 최적화하려는 정책과의 분포 차이가 발생하는 'Off-policy' 편향 문제가 필연적으로 발생합니다. 안정성을 위해 데이터의 신선도를 포기하는 것이 과연 최적의 학습 방법이라고 단언할 수 있습니까?

[Debater]: Experience Replay가 데이터 효율성을 제공한다고 하셨는데, 수백만 개의 트랜지션을 저장하기 위해 소모되는 막대한 메모리 점유는 '하드웨어 효율성' 측면에서 정당화가 됩니까? 또한, 희소 샘플의 재사용성을 강조하셨으나, Prioritized Experience Replay가 아닌 일반적인 무작위 샘플링 환경에서 그 희소한 데이터가 유의미한 확률로 추출되어 학습에 기여할 수 있다고 보시는 건가요? 무엇보다 Experience Replay는 강화학습의

In [ ]:
print(final_status)


LOSS


In [9]:
print(evaluation_logs)

[{'turn': 1, 'score_now': 54, 'reason': "i.i.d. 가정 충족과 Moving Target 문제 해결을 통한 수렴 안정성 확보라는 강화학습의 핵심 논리를 전문적인 용어로 정확하게 설명하였습니다. 논리적 흠결이 없으며 난이도 '상'의 기준을 잘 충족합니다."}, {'turn': 2, 'score_now': 57, 'reason': "비동기 병렬 방식의 하드웨어 자원 효율성 문제를 지적하며 Experience Replay의 샘플 재사용 및 분산 억제 효과를 전문적인 용어로 논리적으로 옹호했습니다. 다만, 난이도가 '상'인 상황에서 공격자가 제기한 'Off-policy 편향'이라는 핵심적인 취약점에 대해 기술적 반증을 제시하지 않고, 단순히 이득이 더 크다는 비교 우위론적 주장에 그친 점은 논리적 완결성 측면에서 감점 요인입니다."}, {'turn': 3, 'score_now': 60, 'reason': "사용자는 상대방이 제기한 세 가지 비판(하드웨어 효율성, 희소 샘플의 확률적 문제, Deadly Triad의 모순)에 대해 논리적으로 대응했습니다. 메모리 비용과 병렬 에이전트 운용 비용을 비교하여 하드웨어 효율성을 정당화한 점과 Target Network를 통한 수렴 안정성 확보를 언급한 점은 타당합니다. 하지만 난이도가 '상'임을 고려할 때, '희소 샘플의 낮은 추출 확률'에 대한 구체적인 반박 대신 '일반화'라는 장점으로 논점을 회피(Red Herring)한 점과, Deadly Triad 중 오프-폴리시 부트스트래핑이 발생하는 이론적 모순을 해결할 구체적인 논리(예: 준정적인 타겟 설정의 수리적 의미 등)가 부족하여 높은 점수를 부여하기 어렵습니다."}, {'turn': 4, 'score_now': 64, 'reason': "사용자는 Stale Data와 Target Network의 지연 현상을 각각 정규화와 안정성 기준점이라는 가치로 치밀하게 재정의하여 방어했습니다. 그러나 난이도가 '상'임을 감안할 때, 공격 측이 제기한 

In [10]:
print(summary_context)

{'summary_text': "이번 토론은 DQN의 핵심인 Experience Replay와 Target Network의 필요성을 주제로 진행되었으며, 사용자님은 논리적 일관성을 유지하다 마지막에 아쉽게 패배(LOSS)하셨습니다. 초기 발언에서 **'i.i.d. 가정 충족'**과 **'Moving Target 문제'**를 정확히 지적하며 수렴 안정성을 논리적으로 방어하신 점은 매우 인상적이었습니다. 특히 상대방의 비판에 맞서 **'Stale Data가 오히려 Local Minima 함몰을 막는 정규화(Regularization) 역할을 수행한다'**는 독창적인 관점을 제시하며 높은 수준의 기술적 통찰력을 보여주셨습니다. 그러나 상대방이 마지막에 제기한 **'PPO/TRPO의 신뢰 구역(Trust Region)을 통한 안정성 확보'** 및 **'벨만 연산의 통계적 오류'**에 대해 구체적인 재반박을 포기하고 비격식적인 태도로 토론을 마무리한 것이 패배의 결정적 요인이 되었습니다. 고난도 토론일수록 마지막까지 기술적 근거를 바탕으로 논리를 매듭짓는 연습이 필요합니다.", 'key_takeaways': ['Deadly Triad (함수 근사, 부트스트래핑, 오프-폴리시의 결합)', 'Catastrophic Forgetting (데이터 상관관계로 인한 파괴적 망각)', 'Distribution Shift (과거와 현재 정책 간의 데이터 분포 차이)']}


In [11]:
print(history)

[{'role': 'system', 'content': "지금부터 DQN(Deep Q-Network)의 핵심 메커니즘인 'Experience Replay'와 'Target Network'의 필연성에 대한 심화 비판 토론을 시작하겠습니다. 본 세션은 총 5턴 동안 진행되며, 난이도 '상'의 엄격한 기준으로 당신의 논리를 검증할 것입니다. 수식의 단순 나열보다는 개념 간의 인과관계를 중점적으로 평가하며, 작은 논리적 비약도 즉각적인 반박의 대상이 됩니다. 강화학습의 고질적인 불안정성 문제를 해결하기 위해 왜 이 두 기능이 필수적인지, 그 당위성을 입증하며 첫 발언을 시작해 주십시오.", 'turn': 'Opening'}, {'role': 'user', 'content': '강화학습에서 발생하는 데이터 간의 시간적 상관관계를 끊어 i.i.d. 가정을 충족시키고, 학습 목표값이 업데이트마다 변하는 Moving Target 문제를 고정하여 네트워크의 수렴 안정성을 확보하기 위해 두 메커니즘은 필수적입니다.', 'turn': 1}, {'role': 'assistant', 'content': "i.i.d. 가정을 충족시키기 위해 Experience Replay가 '필수적'이라고 하셨는데, 그렇다면 별도의 버퍼 없이 병렬 에이전트를 활용해 데이터 상관관계를 깨는 A3C 같은 비동기 방식은 어떻게 설명하실 겁니까? 또한, Replay Buffer에서 샘플링된 데이터는 결국 과거 정책에 의해 수집된 것이므로 현재 최적화하려는 정책과의 분포 차이가 발생하는 'Off-policy' 편향 문제가 필연적으로 발생합니다. 안정성을 위해 데이터의 신선도를 포기하는 것이 과연 최적의 학습 방법이라고 단언할 수 있습니까?", 'turn': 1}, {'role': 'user', 'content': '비동기 병렬 방식은 하드웨어 자원의 비효율적 소모를 통해 상관관계를 간접적으로 완화할 뿐이며, 데이터의 신선도보다 Experience Replay를 통한 희소 샘플의 재사용성과 샘플링 분산 억제가 